In [1]:
!pip --quiet install efficientnet -q

In [2]:
import os
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
import cv2,random,math,warnings
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import random,glob
warnings.filterwarnings('ignore')

2022-10-09 17:18:00.466939: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-10-09 17:18:00.467105: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 42
#seed_everything(seed)

In [4]:
train = pd.read_csv('../input/datathonindoml-2022/train_labels.csv')
test = pd.read_csv('../input/datathonindoml-2022/sample_submission.csv')
train.head()

,id,label
0,0,1
1,1,13
2,2,13
3,3,14
4,4,6


In [5]:
train.shape, test.shape

((16000, 2), (5, 2))

In [6]:
def give_path(row, test=False):
    if test:
        return '../input/indoml-t-dataset/test/' + row + '.jpeg'
    return '../input/indoml-t-dataset/train/' + row + '.jpeg'

In [7]:
train['file_path'] = train['id'].apply(lambda x : give_path(str(x)))
test['file_path'] = test['id'].apply(lambda x : give_path(str(x), True))

In [8]:
train.head()

,id,label,file_path
0,0,1,../input/indoml-t-dataset/train/0.jpeg
1,1,13,../input/indoml-t-dataset/train/1.jpeg
2,2,13,../input/indoml-t-dataset/train/2.jpeg
3,3,14,../input/indoml-t-dataset/train/3.jpeg
4,4,6,../input/indoml-t-dataset/train/4.jpeg


In [9]:
def auto_select_accelerator():
    """
    Reference: 
        * https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
        * https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training
    """
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='png'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(image):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_brightness(image, max_delta = 0.1)
#         image = tf.image.random_saturation(image, lower = 0.75, upper = 1.5)
#         image = tf.image.random_hue(image, max_delta = 0.15)
#         image = tf.image.random_contrast(image, lower = 0.75, upper = 1.5)
        image = tf.image.rot90(image)
        image = tf.image.transpose(image)
        
        return image
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [10]:
COMPETITION_NAME = "indoml-t-dataset"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 8
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

2022-10-09 17:18:06.854813: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-09 17:18:06.857650: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-10-09 17:18:06.857691: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-09 17:18:06.857747: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (1e420f0e98e6): /proc/driver/nvidia/version does not exist
2022-10-09 17:18:06.861053: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Running on TPU: grpc://10.0.0.2:8470
Running on 8 replicas


In [11]:
paths = []
labels = []
for i in range(len(train['file_path'].values)):
    name = train['id'].values[i]
    label = train['label'].values[i]
    paths.append(GCS_DS_PATH + '/train/' + str(name) + '.jpeg')
    labels.append(label)

In [12]:
one_hot = True
if one_hot:
    labels = tf.keras.utils.to_categorical(labels , num_classes = 16)

In [13]:
len(paths) , len(labels)

(16000, 16000)

In [14]:
train_paths,valid_paths,train_labels,valid_labels = train_test_split(paths , labels , stratify = labels , test_size = 0.1 , random_state = 42)

In [15]:
train['label'].value_counts()

8     1059
6     1044
0     1032
12    1020
5     1012
11    1011
13    1007
9     1000
3      993
14     992
15     988
10     974
1      969
2      968
4      966
7      965
Name: label, dtype: int64

In [16]:
temp = os.listdir('../input/indoml-t-dataset/test/')
test_paths = []
for i in range(len(temp)):
    name = temp[i]
    test_paths.append(GCS_DS_PATH + '/test/' + str(name))

In [17]:
# Build the tensorflow datasets
IMSIZES = (224, 240, 260, 300, 380, 456, 512, 600)
im_size = 1024

decoder = build_decoder(with_labels=True, target_size=(im_size, im_size))
test_decoder = build_decoder(with_labels=False, target_size=(im_size, im_size))

train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False
)

test_decoder = build_decoder(with_labels=False, target_size=(im_size, im_size))
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

In [18]:
def get_model(name):
    if name == 'effnetb0':
        with strategy.scope():
            base_model = efn.EfficientNetB0(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'effnetb1':
        with strategy.scope():
            base_model = efn.EfficientNetB1(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'effnetb2':
        with strategy.scope():
            base_model = efn.EfficientNetB2(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'effnetb3':
        with strategy.scope():
            base_model = efn.EfficientNetB3(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'effnetb4':
        with strategy.scope():
            base_model = efn.EfficientNetB4(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'effnetb5':
        with strategy.scope():
            base_model = efn.EfficientNetB5(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'effnetb6':
        with strategy.scope():
            base_model = efn.EfficientNetB6(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'effnetb7':
        with strategy.scope():
            base_model = efn.EfficientNetB7(input_shape=(im_size, im_size, 3), weights='noisy-student', include_top = False)
    elif name == 'resnet':
        with strategy.scope():
            base_model = ResNet50(input_shape=(im_size, im_size, 3), weights='imagenet', include_top = False)
    elif name == 'xception':
        with strategy.scope():
            base_model = Xception(input_shape=(im_size, im_size, 3), weights='imagenet', include_top = False)
    elif name == 'inception':
        with strategy.scope():
            base_model = InceptionV3(input_shape=(im_size, im_size, 3), weights='imagenet', include_top = False)
    elif name == 'inceptionresnet':
        with strategy.scope():
            base_model = InceptionResNetV2(input_shape=(im_size, im_size, 3), weights='imagenet', include_top = False)
        
    with strategy.scope():
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(16, activation='softmax')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr = 1e-4),
            loss= tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.001) , #'sparse_categorical_crossentropy',
            metrics=["accuracy"])
    return model

In [19]:
model_name = "effnetb4"
model = get_model(model_name)
model.summary()

71680000/71678424 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b4 (Functional) (None, 32, 32, 1792)      17673816  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                28688     
Total params: 17,702,504
Trainable params: 17,577,304
Non-trainable params: 125,200
_________________________________________________________________


In [20]:
train_paths = np.array(train_paths)
steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'{model_name}.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience = 3, min_lr = 1e-7, mode='min')

In [21]:
history = model.fit(
    train_dataset,  
    epochs = 20,
    verbose = 1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

Epoch 1/20
225/225 [==============================] - 686s 2s/step - loss: 2.0585 - accuracy: 0.3728 - val_loss: 1.4820 - val_accuracy: 0.5431
Epoch 2/20
225/225 [==============================] - 238s 1s/step - loss: 1.2246 - accuracy: 0.6295 - val_loss: 1.1754 - val_accuracy: 0.6413
Epoch 3/20
225/225 [==============================] - 240s 1s/step - loss: 1.0336 - accuracy: 0.6927 - val_loss: 0.8364 - val_accuracy: 0.7512
Epoch 4/20
225/225 [==============================] - 240s 1s/step - loss: 0.8761 - accuracy: 0.7401 - val_loss: 0.7778 - val_accuracy: 0.7688
Epoch 5/20
225/225 [==============================] - 239s 1s/step - loss: 0.7782 - accuracy: 0.7637 - val_loss: 0.7067 - val_accuracy: 0.7950
Epoch 6/20
225/225 [==============================] - 238s 1s/step - loss: 0.7020 - accuracy: 0.7881 - val_loss: 0.6897 - val_accuracy: 0.8006
Epoch 7/20
225/225 [==============================] - 239s 1s/step - loss: 0.6439 - accuracy: 0.8049 - val_loss: 0.6819 - val_accuracy: 0.8087

KeyboardInterrupt: 

In [22]:
model.load_weights(f'./{model_name}.h5')
preds = model.predict(dtest, verbose=1)

15/15 [==============================] - 48s 3s/step


In [23]:
val_preds = model.predict(valid_dataset, verbose=1)
val_preds = np.round(val_preds, decimals = 0)
print(classification_report(valid_labels , val_preds))

25/25 [==============================] - 18s 587ms/step
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       103
           1       0.80      0.76      0.78        97
           2       0.96      0.99      0.97        97
           3       0.95      0.89      0.92        99
           4       0.94      0.90      0.92        97
           5       0.72      0.66      0.69       101
           6       0.91      0.92      0.92       104
           7       0.91      0.84      0.87        97
           8       0.90      0.93      0.92       106
           9       0.90      0.85      0.88       100
          10       0.78      0.81      0.80        97
          11       0.95      0.76      0.85       101
          12       0.77      0.77      0.77       102
          13       0.83      0.74      0.79       101
          14       0.98      0.92      0.95        99
          15       0.95      0.87      0.91        99

   micro avg       0.88 

In [24]:
sub = pd.DataFrame()
sub['id'] = test_paths
sub['id'] = sub['id'].apply(lambda x: int(x.split('.')[-2].split('/')[-1]))
sub['label'] = np.argmax(preds, axis=-1)
sub.head()

,id,label
0,17981,5
1,18140,4
2,18548,0
3,18236,2
4,18144,9


In [25]:
sub.to_csv('sub.csv', index=False)